In [1]:
# !pip install mediapipe opencv-python
# !pip3 install tensorflowjs
# !pip install protobuf==3.20.*
# !pip uninstall mediapipe
# !pip install mlserver==1.0.1 grpcio==3.20.1

In [2]:
import cv2
import mediapipe as mp
import tensorflow as tf
import csv
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# import tensorflowjs as tfjs
# import tensorflowjs as tfjs

2023-05-30 20:32:15.123723: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:32:15.212847: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:32:15.213961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:32:16.959098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
# cap = cv2.VideoCapture(0)
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#   while cap.isOpened():
#     success, image = cap.read()
#     if not success:
#       print("Unable to record")
#       continue
#     image.flags.writeable = False
#     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#     results = pose.process(image)

#     image.flags.writeable = True
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())

#     cv2.imshow('MediaPipe Pose', cv2.flip(image,1))

#     if cv2.waitKey(5) & 0xFF == ord('q'):
#       break
# cap.release()
# cv2.destroyAllWindows()

In [5]:
# def get_center_point(landmarks, left_bodypart, right_bodypart):
#     left = tf.gather(landmarks, left_bodypart.value, axis=1)
#     right = tf.gather(landmarks, right_bodypart.value, axis=1)
#     center = left * 0.5 + right * 0.5
#     return center


# def get_pose_size(landmarks, torso_size_multiplier=2.5):
#     hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                  BodyPart.RIGHT_HIP)

#   # Shoulders center
#     shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
#                                       BodyPart.RIGHT_SHOULDER)

#   # Torso size as the minimum body size
#     torso_size = tf.linalg.norm(shoulders_center - hips_center)

#   # Pose center
#     pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                      BodyPart.RIGHT_HIP)
#     pose_center_new = tf.expand_dims(pose_center_new, axis=1)
#   # Broadcast the pose center to the same size as the landmark vector to
#   # perform substraction
#     pose_center_new = tf.broadcast_to(pose_center_new,
#                                     [tf.size(landmarks) // (17*2), 17, 2])

#   # Dist to pose center
#     d = tf.gather(landmarks - pose_center_new, 0, axis=0,
#                 name="dist_to_pose_center")
#   # Max dist to pose center
#     max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

#   # Normalize scale
#     pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

#     return pose_size


# def normalize_pose_landmarks(landmarks):
#     pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                  BodyPart.RIGHT_HIP)
#     pose_center = tf.expand_dims(pose_center, axis=1)
#   # Broadcast the pose center to the same size as the landmark vector to perform
#   # substraction
#     pose_center = tf.broadcast_to(pose_center, 
#                                 [tf.size(landmarks) // (17*2), 17, 2])
#     landmarks = landmarks - pose_center

#   # Scale the landmarks to a constant pose size
#     pose_size = get_pose_size(landmarks)
#     landmarks /= pose_size

#     return landmarks


# def landmarks_to_embedding(landmarks_and_scores):
#     reshaped_inputs = keras.layers.Reshape((33, 3))(landmarks_and_scores)

#   # Normalize landmarks 2D
#     landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])

#   # Flatten the normalized landmark coordinates into a vector
#     embedding = keras.layers.Flatten()(landmarks)

#     return embedding

In [6]:
# def get_center_point(landmarks, left_bodypart, right_bodypart):
#     left = tf.gather(landmarks, left_bodypart, 1)
#     right = tf.gather(landmarks, right_bodypart, 1)
#     center = tf.add(tf.mul(left, 0.5), tf.mul(right, 0.5))
#     return center
  

# def get_pose_size(landmarks, torso_size_multiplier = 2.5):
#     hips_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     shoulders_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_SHOULDER,
#       POINTS.RIGHT_SHOULDER
#     )
#     torso_size = tf.norm(tf.sub(shoulders_center, hips_center))
#     pose_center_new = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     pose_center_new = tf.expandDims(pose_center_new, 1)

# #     // pose_center_new = tf.broadcastTo(pose_center_new, [1, 17, 2]);
# #     // return: shape(17,2)
#     d = tf.gather(tf.sub(landmarks, pose_center_new), 0, 0)
#     max_dist = tf.max(tf.norm(d, "euclidean", 0))

# #     // normalize scale
#     pose_size = tf.maximum(
#       tf.mul(torso_size, torso_size_multiplier),
#       max_dist
#     )
#     return pose_size

# def normalize_pose_landmarks(landmarks):
#     pose_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     pose_center = tf.expandDims(pose_center, 1)
# #     // pose_center = tf.broadcastTo(pose_center, [1, 17, 2])
#     landmarks = tf.sub(landmarks, pose_center)

#     pose_size = get_pose_size(landmarks)
#     landmarks = tf.div(landmarks, pose_size)
#     return landmarks

# def landmarks_to_embedding(landmarks):
# #     // normalize landmarks 2D
#     landmarks = normalize_pose_landmarks(tf.expandDims(landmarks, 0))
#     embedding = tf.reshape(landmarks, [1, 99])
# #     // console.log(embedding.array().then((data) => console.log(data)));
#     return embedding
  

In [7]:
# Run this to generate pose coordinate
# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
#                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [8]:
# def get_center_point(pose_landmarks):
#     left_hip = pose_landmarks.landmark[23]
#     right_hip = pose_landmarks.landmark[24]
#     hip_center_x = (left_hip.x + right_hip.x)*0.5;
#     hip_center_y = (left_hip.y + right_hip.y)*0.5;
                    
#     left_shoulder = pose_landmarks.landmark[11]
#     right_shoulder = pose_landmarks.landmark[12]
#     shoulder_center_x = (left_shoulder.x + right_shoulder.x)*0.5;
#     shoulder_center_y = (left_shoulder.y + right_shoulder.y)*0.5;
                    
#     torso_center_x = (hip_center_x + shoulder_center_x )*0.5;
#     torso_center_y = (hip_center_y + shoulder_center_y )*0.5;

#     return torso_center_x, torso_center_y

In [9]:
# def normalize(landmarks,pose_center_new):
#     d = tf.gather(landmarks - pose_center_new, 0, axis=0,
#                 name="dist_to_pose_center")
#       # Max dist to pose center
#     max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

#       # Normalize scale
#     pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

In [10]:

# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
#                     torso_center_x, torso_center_y = get_center_point(pose_landmarks)
#                     print(torso_center_x, torso_center_y)
# #                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x - torso_center_x, lmk.y - torso_center_y, lmk.z] for lmk in pose_landmarks.landmark]

#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [11]:

# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
# #                     torso_center_x, torso_center_y = get_center_point(pose_landmarks)
# #                     print(torso_center_x, torso_center_y)
# #                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x , lmk.y , lmk.z] for lmk in pose_landmarks.landmark]
#                     pose_landmarks = landmarks_to_embedding(pose_landmarks)
#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [12]:
df = pd.read_csv('pose_coordinate.csv',names=[x for x in range(101)])

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,girl1_chair070.jpg,chair,161.93193,91.50034,-113.45271,161.39712,86.31226,-106.83022,161.58342,86.05865,...,151.52425,145.37344,263.04284,35.51275,164.25312,266.42942,134.10185,161.96825,272.50385,2.88960
1,girl1_chair070_flipped.jpg,chair,139.86855,91.82668,-104.83980,143.78042,87.16641,-108.79897,145.48917,87.27382,...,48.08073,150.45469,237.26957,172.43419,138.36648,272.82157,21.53601,134.58295,257.23835,168.02629
2,girl1_chair075.jpg,chair,167.14772,99.77740,-93.39951,166.52445,95.02889,-87.08542,166.49821,94.65935,...,133.62957,146.21243,266.85129,35.65139,168.46964,268.35528,129.69736,168.64370,275.69879,28.96146
3,girl1_chair075_flipped.jpg,chair,136.16109,100.78100,-76.01230,139.12286,96.90815,-79.26770,140.43436,96.86092,...,-41.07904,145.33850,236.36127,128.01884,133.34532,273.72544,-64.76517,132.31215,257.97881,127.25959
4,girl1_chair076.jpg,chair,168.41848,104.34607,-96.49125,168.76974,99.73064,-89.25885,169.08849,99.53190,...,119.36260,146.60322,268.53570,22.31681,169.18559,264.90762,118.56466,172.15129,276.03448,13.66442


In [14]:
df.shape

(2096, 101)

In [15]:
df.columns

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100],
      dtype='int64', length=101)

In [16]:
df.drop(0,axis=1,inplace=True)

In [17]:
df.iloc[:,1:].values

array([[ 161.93193,   91.50034, -113.45271, ...,  161.96825,  272.50385,
           2.8896 ],
       [ 139.86855,   91.82668, -104.8398 , ...,  134.58295,  257.23835,
         168.02629],
       [ 167.14772,   99.7774 ,  -93.39951, ...,  168.6437 ,  275.69879,
          28.96146],
       ...,
       [ 160.03075,  101.21098, -275.6927 , ...,  169.06106,   76.79547,
          44.96757],
       [ 135.92988,   85.11297, -182.00737, ...,  134.54167,  275.22699,
          46.33296],
       [ 164.71746,   97.26677, -219.01392, ...,  180.18463,  104.69069,
         416.26228]])

In [18]:
x = df.iloc[:,1:].values

In [19]:
lis = [x for x in df[1]]

In [20]:
le = LabelEncoder()

In [21]:
le.fit(df[1])

LabelEncoder()

In [22]:
lis = le.transform(df[1])

In [23]:
df[1].unique()

array(['chair', 'cobra', 'dog', 'no_pose', 'shoudler_stand', 'traingle',
       'tree', 'warrior'], dtype=object)

In [24]:
lis

array([0, 0, 0, ..., 7, 7, 7])

In [25]:
import keras

In [26]:
y = keras.utils.to_categorical(lis,num_classes=9)

In [27]:
y.shape

(2096, 9)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42)

In [30]:
x_train.shape

(1572, 99)

In [31]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

In [32]:
x.shape

(2096, 99)

In [33]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=x.shape[1]))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25600     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 9)                 585       
                                                                 
Total params: 67,337
Trainable params: 67,337
Non-traina

In [34]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=20, min_delta =0.1 ,restore_best_weights=True)

In [35]:
history = model.fit(x_train, y_train, batch_size=25,epochs=1000,callbacks=[callback])

Epoch 1/1000
63/63 [==============================] - 2s 8ms/step - loss: 23.0216 - accuracy: 0.4294
Epoch 2/1000
63/63 [==============================] - 0s 6ms/step - loss: 4.2347 - accuracy: 0.6088
Epoch 3/1000
63/63 [==============================] - 0s 6ms/step - loss: 2.6709 - accuracy: 0.6838
Epoch 4/1000
63/63 [==============================] - 0s 6ms/step - loss: 1.7223 - accuracy: 0.7316
Epoch 5/1000
63/63 [==============================] - 0s 6ms/step - loss: 1.3988 - accuracy: 0.7449
Epoch 6/1000
63/63 [==============================] - 0s 6ms/step - loss: 1.1487 - accuracy: 0.7729
Epoch 7/1000
63/63 [==============================] - 0s 6ms/step - loss: 0.8692 - accuracy: 0.8174
Epoch 8/1000
63/63 [==============================] - 0s 6ms/step - loss: 0.6839 - accuracy: 0.8416
Epoch 9/1000
63/63 [==============================] - 0s 6ms/step - loss: 0.5835 - accuracy: 0.8524
Epoch 10/1000
63/63 [==============================] - 0s 6ms/step - loss: 0.7582 - accuracy: 0.825

In [36]:
model.save("model")

2023-05-30 20:32:46.099272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:32:46.139469: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-05-30 20:32:46.667256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:32

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [37]:
model.predict(x_test)

17/17 [==============================] - 0s 4ms/step


array([[9.99713182e-01, 2.91081747e-13, 7.87485233e-22, ...,
        2.49618215e-10, 2.86793365e-04, 6.05734924e-23],
       [1.02918683e-12, 1.07274745e-09, 9.99997079e-01, ...,
        7.54216203e-11, 2.66668359e-08, 3.62485281e-12],
       [1.28606274e-14, 6.47582031e-13, 9.99999940e-01, ...,
        1.53624794e-17, 3.05535069e-10, 8.96806150e-20],
       ...,
       [1.06840209e-26, 1.42640563e-19, 1.00000000e+00, ...,
        2.47363176e-26, 7.71170162e-17, 1.47709567e-26],
       [5.97830621e-34, 5.18588126e-28, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.63034860e-07, 9.04343307e-01, 2.76073824e-05, ...,
        3.59806000e-12, 9.56285670e-02, 5.93206817e-10]], dtype=float32)

In [38]:
model.evaluate(x_test,y_test)

17/17 [==============================] - 0s 3ms/step - loss: 0.1902 - accuracy: 0.9656


[0.19019049406051636, 0.9656488299369812]

In [39]:
model.get_weights()

[array([[ 0.00362732, -0.03299519, -0.06602887, ..., -0.12187836,
         -0.08394112,  0.0406397 ],
        [-0.13075754, -0.01511293,  0.06115869, ..., -0.06858579,
         -0.01144459, -0.02603509],
        [-0.01057925,  0.0130458 , -0.11499365, ..., -0.11938981,
          0.07120433, -0.07885953],
        ...,
        [-0.09669636,  0.07845997, -0.0203625 , ..., -0.1224263 ,
          0.03129628, -0.08159684],
        [-0.12634106,  0.05677187,  0.09904908, ..., -0.11248008,
         -0.062567  , -0.09240075],
        [ 0.10425552, -0.07516393, -0.02837862, ..., -0.13096017,
         -0.04646995,  0.02143036]], dtype=float32),
 array([-3.88248302e-02, -2.23291274e-02, -4.53516915e-02,  1.42791336e-02,
        -2.94715445e-02,  8.56748130e-03, -1.38189588e-02, -2.81076394e-02,
        -1.56145366e-02, -7.40728993e-03, -4.81512398e-02,  2.01320499e-02,
        -5.34381298e-03, -1.73008740e-02, -2.15249360e-02, -1.74675882e-02,
         1.95867158e-02, -7.76576623e-03, -9.40593425e

In [40]:
model.save("model")

2023-05-30 20:32:49.621296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:32:49.659354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-05-30 20:32:50.221143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:32

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [41]:
np.expand_dims(x_test[0],axis=0).shape

(1, 99)

In [42]:
import pickle

In [43]:
pickle.dump(model, open('model_pkl.pkl','wb'))

In [44]:
# model_pkl = pickle.load(open('model_pkl.pkl','rb'))

In [45]:
model.predict(np.expand_dims(x_test[0],axis=0))

1/1 [==============================] - 0s 83ms/step


array([[9.9971324e-01, 2.9108457e-13, 7.8749129e-22, 4.7510197e-17,
        7.9484831e-17, 3.5542586e-10, 2.4961919e-10, 2.8679392e-04,
        6.0573732e-23]], dtype=float32)

In [46]:
# mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [47]:
le.inverse_transform([0])

array(['chair'], dtype=object)

In [48]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Unable to record")
            continue
        image.flags.writeable = False
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
        results = pose.process(image)
        pose_landmarks = results.pose_landmarks
        output_frame = image.copy()
        pose_name = 'Unidentified'
        if pose_landmarks is not None:
#             torso_center_x, torso_center_y = get_center_point(pose_landmarks)
            vector = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
            frame_height, frame_width = output_frame.shape[:2]

            vector *= np.array([frame_width, frame_height, frame_width])
            vector = vector.flatten()
            vector = np.expand_dims(vector,axis=0)

            prediction = model.predict((vector))

            index = np.argmax(prediction)

            pose_name = le.inverse_transform([index])
#             print("pose name:",pose_name)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())

        image = cv2.flip(image,1)

        #Rectangle
        start_point = (0, 0)
        end_point = (300, 100)
        color = (0, 0, 0)
        thickness = -1
        image = cv2.rectangle(image, start_point, end_point, color, thickness)
        #write
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (25, 50)
        fontScale = 1
        color = (255, 255, 255)
        thickness = 2

        image = cv2.putText(image, pose_name[0], org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow('MediaPipe Pose',image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1/1 [==============================] - 0s 53ms/step


In [49]:
#EOF

In [50]:
df[1].unique()

array(['chair', 'cobra', 'dog', 'no_pose', 'shoudler_stand', 'traingle',
       'tree', 'warrior'], dtype=object)

In [51]:
pose_dict={};

In [52]:
for pose in df[1].unique():
    pose_dict[str(le.transform([pose])[0])] = pose

In [53]:
le.transform(['chair'])

array([0])

In [54]:
pose_dict

{'0': 'chair',
 '1': 'cobra',
 '2': 'dog',
 '3': 'no_pose',
 '4': 'shoudler_stand',
 '5': 'traingle',
 '6': 'tree',
 '7': 'warrior'}

In [55]:
import json

In [56]:
with open('pose_labels.txt','w') as f:
    f.write(json.dumps(pose_dict))

In [57]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

print('Model size: %dKB' % (len(tflite_model) / 1024))

with open('pose_classifier.tflite', 'wb') as f:
  f.write(tflite_model)

2023-05-30 20:32:57.951076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:32:57.987003: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-05-30 20:32:58.398259: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:32

INFO:tensorflow:Assets written to: /tmp/tmpme5olobj/assets


INFO:tensorflow:Assets written to: /tmp/tmpme5olobj/assets


Model size: 71KB


2023-05-30 20:33:00.544094: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-30 20:33:00.544167: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-30 20:33:00.545032: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpme5olobj
2023-05-30 20:33:00.547453: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-30 20:33:00.547515: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpme5olobj
2023-05-30 20:33:00.561122: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-05-30 20:33:00.564835: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-05-30 20:33:00.724716: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpme5olobj
2023-05

In [58]:
# with open('pose_labels.txt', 'w') as f:
#   f.write('\n'.join(df[1]))

In [59]:
# from zipfile import ZipFile

In [60]:
# with ZipFile('model.zip', 'w') as zipObj:
#     zipObj.write('pose_classifier.tflite')
#     zipObj.write('pose_labels.txt')
#     zipObj.close()

In [61]:
# tfjs.converters.save_keras_model(model,'model')

In [62]:
# !zip pose_classifier.zip pose_labels.txt pose_classifier.tflite

In [63]:
# try:
#   from google.colab import files
#   files.download('pose_classifier.zip')
# except:
#   pass

In [64]:
model.predict(x_test)

17/17 [==============================] - 0s 3ms/step


array([[9.99713182e-01, 2.91081747e-13, 7.87485233e-22, ...,
        2.49618215e-10, 2.86793365e-04, 6.05734924e-23],
       [1.02918683e-12, 1.07274745e-09, 9.99997079e-01, ...,
        7.54216203e-11, 2.66668359e-08, 3.62485281e-12],
       [1.28606274e-14, 6.47582031e-13, 9.99999940e-01, ...,
        1.53624794e-17, 3.05535069e-10, 8.96806150e-20],
       ...,
       [1.06840209e-26, 1.42640563e-19, 1.00000000e+00, ...,
        2.47363176e-26, 7.71170162e-17, 1.47709567e-26],
       [5.97830621e-34, 5.18588126e-28, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.63034860e-07, 9.04343307e-01, 2.76073824e-05, ...,
        3.59806000e-12, 9.56285670e-02, 5.93206817e-10]], dtype=float32)

In [65]:
model.evaluate(x_test, y_test)

17/17 [==============================] - 0s 7ms/step - loss: 0.1902 - accuracy: 0.9656


[0.19019049406051636, 0.9656488299369812]